# Flywheel Software Development Kit (SDK) Analysis Derivatives Upload Example (RISE Project)
**Date modified:** 16-Aug-2022
**Authors:** Amy Hegarty, Intermountain Neuroimaging Consortium

**Description:**
This workbook outlines an example upload for previously generated analysis results on a remote system. For example, you want to save the details of an analysis ran on your local computer within flywheel's project heiarchy. This example we will show how to upload a test job, followed by a working example to upload an fmriprep analysis. 

## Install and Import Dependencies 

In [ ]:
# Install specific packages required for this notebook
!pip install flywheel-sdk 
!pip install flywheel-gear-toolkit
!pip install pandas

In [ ]:
# Set long timeout window for large uploads
!export FLYWHEEL_SDK_REQUEST_TIMEOUT=600

In [ ]:
 # Import packages
from getpass import getpass
from pathlib import Path
import os, sys
import logging
import pandas as pd
import flywheel
from flywheel_gear_toolkit.utils.zip_tools import unzip_archive, zip_output
from zip_htmls import zip_htmls  ## note, this is a local python script that needs
                                 ## to be placed in the current working directory to be found and imported
from datetime import datetime as dt

## Getting Started

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [ ]:
# create the flywheel client, using CLI credentials 
#  (you need to be logged into Flywheel from the CLI to use this method)
fw = flywheel.Client()

# who am I logged in as?
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Add Supporting Functions 

In [ ]:
# supporting functions
def analysis_exists(session, analysis_name, exclude_list=[]):
    # Returns True if analysis already exists with a running or complete status, else false
    # make sure to pass full session object (use fw.get_session(session.id))
    #
    #Get all analyses for the session
    flag=False
    for analysis in session.analyses:
        #only print ones that match the  analysis label
        if analysis_name in analysis.label:
            #filter for only successful job
            analysis_job=analysis.job
            if any(analysis_job.state in string for string in ["complete","running","pending"]):
                if analysis_job.failure_reason is None:
                    flag=True
                    
        #check if session is in exclude list
        if any(session.id in string for string in exclude_list):
            flag=True
    
    return flag

In [ ]:
def printfile(filename):
    print(filename)
    f = open(filename, 'r')
    print(f.read())

## Accessing Your Flywheel Project

In [ ]:
# locate your project using the lookup function
project = fw.lookup('ics/sandbox')

# this will shows all the property available in the project
project.keys()

# list sessions in the project
for session in project.sessions.find():
    print('%s: Subject: %s Session: %s\tScanning Date: %s' % (session.id, session.subject.label, session.label, session.timestamp))
    

In [ ]:
# list all sessions with a specific analysis run / uploaded
analysisname = 'fmriprep'
has_anlys = []
for session in project.sessions.find():
    full_session = fw.get_session(session.id)
    flag = analysis_exists(full_session, analysisname)
    if flag:
        has_anlys.append([session.subject.label,session.label])
        
print(has_anlys)

In [ ]:
# add a new analysis
sess=fw.lookup('ics/sandbox/10462/S1')
analysis = sess.add_analysis(label='test-upload'+dt.now().strftime(" %x %X"))

## Adding Analysis Derivative Datasets To Flywheel
Here are some considerations to keep in mind when uploading analyses computed on a remote system...
1. Include a full copy of all software / scripts used to generate the analysis (if it is not stored in a public, stable repository)
2. Include all configuration settings for the analysis (e.g. command line aurguments)
3. Include any logs or notes regarded the status of the analysis
4. All data files should be zipped into a single outputs folder, preferably in the same directory structure so the derivative dataset can be easily reconstructed at download. (Consider including a list of all files included in the upload and the upload data file size)

In the following example we show an example of best practice to upload an fmriprep analysis which was executed on a remote system. Since fmirprep is a publically availible software, the underlying docker image does not need to be retained. However all scripts including if possible compute configuration, command line aurguments and logs should be included in the analysis upload.

We will include the following files:
- analysis_configuration.txt 
- analysis_information.txt
- bids-fmriprep.zip
- logs.zip
- run_script[.sh,.zip]
- report.html.zip

Lets look at some examples of these upload files...

In [ ]:
# Look at the contents of "Analysis Configuration"
path = '<path-to-files>'
printfile(path+'analysis_configuration.txt')

In [ ]:
# Look at the contents of "Analysis Information"
printfile(path+'analysis_information.txt')

In [ ]:
# Look at the contents of "run_script"
printfile(path+'run_script.sh')

Next we zip the output directory, logs, and any other directories you choose to upload...

In [ ]:
# select a session to attach fmriprep
sess = fw.lookup('ics/sandbox/10462/S1')

# locate fmriprep directory of interest
subid = sess.subject.label
sesid = sess.label

# we need to split the 'base' path and 'relative' path so that we can zip 
#    the contents of the fmriprep directory in a way that it can 
#    be easily reconstructed at download
basepath = '<path_to_local_directory>'
relativepath = 'fmriprep/sub-'+subid+'/ses-'+sesid
uploadspath = basepath + '/uploads/'

# make new upload path if it doesn't exist
os.system('mkdir -p '+uploadspath)

#zip output directory except for scratch
log.info('Zipping contents of directory %s', basepath+'/'+relativepath)
zip_output(basepath,relativepath,uploadspath+'/bids-fmriprep.zip',exclude_files=['scratch'])

In [ ]:
#zip logs
logspath = 'path_to_analysis_logs_if_presnt>
log.info('Zipping contents of directory %s', basepath+'/'+logspath)
zip_output(basepath,logspath,uploadspath+'/logs.zip')

In [ ]:
# upload fmriprep report as html
suffix = 'upload' # this suffix is attached to html name, typically is the jobID
zip_htmls(uploadspath,suffix,relativepath)

Once all analysis related files have been created, you are ready to create a new flywheel analysis and upload all analysis files.

In [ ]:
# create an analysis for the desired flywheel session
analysis = session_object.add_analysis(label='bids-fmriprep: upload '+dt.now().strftime(" %x %X"))
log.info('Creating analysis %s', analysis.label)

# list of all files to attached to analysis 
uploadlist=[
              uploadspath+'/analysis_configuration.txt',
              uploadspath+'/analysis_information.txt',
              uploadspath+'/bids-fmriprep.zip',
              uploadspath+'/logs.zip',
              uploadspath+'/run_script.sh',
              uploadspath+'/report.html.zip'
           ]

for file in uploadlist:
    log.info('Uploading %s', file)
    #upload output file to analysis container
    analysis.upload_output(file)

***Thanks All!*** Check Flywheel to ensure your analysis made it to the correct spot!